# BillSum

In [1]:
!pip install pingouin
!pip install scikit_posthocs
!pip install krippendorff

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 4.1 MB/s eta 0:00:00


In [6]:
import pandas as pd
import re

def process_csv(eval_file, gen_file):
    eval_df = pd.read_csv(eval_file)
    gen_df = pd.read_csv(gen_file)

    # Ensure the dataframes have the same number of rows
    min_rows = min(len(eval_df), len(gen_df))
    eval_df = eval_df.iloc[:min_rows]
    gen_df = gen_df.iloc[:min_rows]

    # Merge the dataframes
    merged_df = pd.merge(eval_df, gen_df[['truncated_input', 'generated_summary']], left_index=True, right_index=True)

    # Add model_name, norm_type, and variant columns from gen_df
    merged_df['model_name'] = gen_df['model_name']
    merged_df['norm_type'] = gen_df['norm_type']
    merged_df['variant'] = gen_df['variant']

    return merged_df

def main():
    evaluation_files = [
        'LN_AttnOnly_gpt4_summary_parsed_evaluations.csv',
        'LN_FFNonly_gpt4_summary_parsed_evaluations.csv',
        'LN_baseModel_gpt4_summary_parsed_evaluations.csv',
        'LN_noNorm_gpt4_summary_parsed_evaluations.csv',
        'RMSN_AttnOnly_gpt4_summary_parsed_evaluations.csv',
        'RMSN_FFNonly_gpt4_summary_parsed_evaluations.csv',
        'RMSN_baseModel_gpt4_summary_parsed_evaluations.csv',
        'RMSN_noNorm_gpt4_summary_parsed_evaluations.csv'
    ]

    generated_summary_files = [
        'LN_AttnOnly_evaluation_data_modified.csv',
        'LN_FFNonly_evaluation_data_modified.csv',
        'LN_baseModel_evaluation_data_modified.csv',
        'LN_noNorm_evaluation_data_modified.csv',
        'RMSN_AttnOnly_evaluation_data_modified.csv',
        'RMSN_FFNonly_evaluation_data_modified.csv',
        'RMSN_baseModel_evaluation_data_modified.csv',
        'RMSN_noNorm_evaluation_data_modified.csv'
    ]

    all_data = pd.concat([process_csv(eval_file, gen_file)
                          for eval_file, gen_file in zip(evaluation_files, generated_summary_files)],
                         ignore_index=True)

    columns_order = ['model_name', 'norm_type', 'variant', 'truncated_input', 'generated_summary',
                     'Overall Score', 'Relevance Score', 'Conciseness Score', 'Fluency Score',
                     'Accuracy Score', 'Coherence Score', 'Overall Feedback', 'Comments on Columns']

    all_data = all_data[columns_order]

    # Truncate generated_summary to 1000 characters
    all_data['generated_summary'] = all_data['generated_summary'].str[:1000]

    # Save the entire dataframe to a single CSV file
    output_file = 'combined_data_truncated.csv'
    all_data.to_csv(output_file, index=False)
    print(f"All data has been saved to '{output_file}' with generated summary truncated to 1000 characters.")

    print(f"The file contains {len(all_data)} rows and {len(all_data.columns)} columns.")
    print("Columns:", ', '.join(all_data.columns))

if __name__ == "__main__":
    main()

All data has been saved to 'combined_data_truncated.csv' with generated summary truncated to 1000 characters.
The file contains 200 rows and 13 columns.
Columns: model_name, norm_type, variant, truncated_input, generated_summary, Overall Score, Relevance Score, Conciseness Score, Fluency Score, Accuracy Score, Coherence Score, Overall Feedback, Comments on Columns


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os

np.random.seed(42)

def load_data(file_path):
    df = pd.read_csv(file_path)
    print(f"Loaded {len(df)} rows of data")
    print(f"Columns: {', '.join(df.columns)}")
    return df

def calculate_statistics(df):
    score_columns = ['Overall Score', 'Relevance Score', 'Conciseness Score',
                     'Fluency Score', 'Accuracy Score', 'Coherence Score']
    return df.groupby(['model_name', 'norm_type', 'variant'])[score_columns].agg(['mean', 'std', 'min', 'max'])

def perform_anova(df, score_column):
    models = df['model_name'].unique()
    data = [df[df['model_name'] == model][score_column] for model in models]
    f_value, p_value = stats.f_oneway(*data)
    return f_value, p_value

def create_collated_boxplots(df, output_dir):
    score_columns = ['Overall Score', 'Relevance Score', 'Conciseness Score',
                     'Fluency Score', 'Accuracy Score', 'Coherence Score']

    plt.figure(figsize=(20, 15))
    for i, score in enumerate(score_columns, 1):
        plt.subplot(2, 3, i)
        sns.boxplot(x='model_name', y=score, hue='norm_type', data=df)
        plt.title(score)
        plt.xticks(rotation=45, ha='right')
        plt.xlabel('')
        if i % 3 != 1:
            plt.ylabel('')
        if i == 3:
            plt.legend(title='Norm Type', bbox_to_anchor=(1.05, 1), loc='upper left')
        else:
            plt.legend([])

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'summarization_collated_boxplots.png'), bbox_inches='tight')
    plt.close()

def main():
    output_dir = 'analysis_results'
    os.makedirs(output_dir, exist_ok=True)

    df = load_data('combined_data_truncated.csv')

    stats_df = calculate_statistics(df)
    stats_df.to_csv(os.path.join(output_dir, 'descriptive_statistics.csv'))

    create_collated_boxplots(df, output_dir)

    score_columns = ['Overall Score', 'Relevance Score', 'Conciseness Score',
                     'Fluency Score', 'Accuracy Score', 'Coherence Score']
    anova_results = []
    for score in score_columns:
        f_value, p_value = perform_anova(df, score)
        anova_results.append({
            'Metric': score,
            'F-value': f_value,
            'p-value': p_value
        })

    anova_df = pd.DataFrame(anova_results)
    anova_df.to_csv(os.path.join(output_dir, 'summarization_anova_results.csv'), index=False)

    print("Analysis complete. Results saved in 'analysis_results' directory.")

if __name__ == "__main__":
    main()

Loaded 200 rows of data
Columns: model_name, norm_type, variant, truncated_input, generated_summary, Overall Score, Relevance Score, Conciseness Score, Fluency Score, Accuracy Score, Coherence Score, Overall Feedback, Comments on Columns
Analysis complete. Results saved in 'analysis_results' directory.


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import kruskal
from scikit_posthocs import posthoc_dunn
import warnings

warnings.filterwarnings('ignore')

# Load the data
df = pd.read_csv('combined_data_truncated.csv')

print("Columns in the CSV file:")
print(df.columns)

# Ensure 'model_name' is treated as a categorical variable
df['model_name'] = df['model_name'].astype('category')

# Define new score columns
score_columns = ['Overall Score', 'Relevance Score', 'Conciseness Score', 'Fluency Score', 'Accuracy Score', 'Coherence Score']

print(f"\nUsing score columns: {score_columns}")

# 1. Effect Size Calculation
def calculate_effect_size(df, metric):
    try:
        f_value, _ = stats.f_oneway(*[group[metric] for name, group in df.groupby('model_name')])
        return f_value / (f_value + df.groupby('model_name').size().iloc[0] - 1)
    except Exception as e:
        print(f"Error calculating effect size for {metric}: {e}")
        return np.nan

effect_sizes = {metric: calculate_effect_size(df, metric) for metric in score_columns}
print("\nEffect Sizes (Eta Squared):")
print(pd.Series(effect_sizes))

# 2. Post-hoc Tests
def perform_tukey_hsd(df, metric):
    try:
        tukey = pairwise_tukeyhsd(df[metric], df['model_name'])
        return pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
    except Exception as e:
        print(f"Error performing Tukey HSD for {metric}: {e}")
        return pd.DataFrame()

tukey_results = {metric: perform_tukey_hsd(df, metric) for metric in score_columns}
print(f"\nTukey HSD Results (example for {score_columns[0]}):")
print(tukey_results[score_columns[0]])

# 3. Correlation Analysis
correlation_matrix = df[score_columns].corr()
print("\nCorrelation Matrix:")
print(correlation_matrix)

# 4. Principal Component Analysis
scaler = StandardScaler()
pca = PCA()
pca_result = pca.fit_transform(scaler.fit_transform(df[score_columns]))
print("\nPCA Explained Variance Ratio:")
print(pca.explained_variance_ratio_)

# 5. Non-parametric Tests
def perform_kruskal_dunn(df, metric):
    try:
        kruskal_result = kruskal(*[group[metric].values for name, group in df.groupby('model_name')])
        if kruskal_result.pvalue < 0.05:
            dunn_result = posthoc_dunn(df, val_col=metric, group_col='model_name', p_adjust='bonferroni')
            return kruskal_result, dunn_result
        return kruskal_result, None
    except Exception as e:
        print(f"Error performing Kruskal-Wallis and Dunn's test for {metric}: {e}")
        return None, None

kruskal_dunn_results = {metric: perform_kruskal_dunn(df, metric) for metric in score_columns}
print(f"\nKruskal-Wallis and Dunn's Test Results (example for {score_columns[0]}):")
print(f"Kruskal-Wallis: {kruskal_dunn_results[score_columns[0]][0]}")
print("Dunn's Test:")
print(kruskal_dunn_results[score_columns[0]][1])

# 6. Visualizations
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap of Metrics')
plt.tight_layout()
plt.savefig('correlation_heatmap_new_metrics.png')
plt.close()

# Violin Plots
plt.figure(figsize=(20, 15))
for i, metric in enumerate(score_columns, 1):
    plt.subplot(2, 3, i)
    sns.violinplot(x='model_name', y=metric, data=df)
    plt.title(metric)
    plt.xticks(rotation=45, ha='right')
    plt.xlabel('')
    if i % 3 != 1:
        plt.ylabel('')
plt.tight_layout()
plt.savefig('violin_plots_new_metrics.png', bbox_inches='tight')
plt.close()

# Radar Chart
def radar_chart(df, metrics):
    means = df.groupby('model_name')[metrics].mean()
    angles = np.linspace(0, 2*np.pi, len(metrics), endpoint=False)
    means = pd.concat([means, means.iloc[:, :1]], axis=1)
    angles = np.concatenate((angles, [angles[0]]))

    fig, ax = plt.subplots(figsize=(14, 10), subplot_kw=dict(projection='polar'))
    for model in means.index:
        values = means.loc[model].values
        ax.plot(angles, values, 'o-', linewidth=2, label=model)
        ax.fill(angles, values, alpha=0.25)
    ax.set_thetagrids(angles[:-1] * 180/np.pi, metrics)
    ax.set_ylim(0, 5)  # Assuming scores are between 0 and 5
    plt.legend(loc='center left', bbox_to_anchor=(1.1, 0.5))
    plt.title("Model Performance Across New Metrics")
    plt.tight_layout()
    plt.savefig('radar_chart_new_metrics.png', bbox_inches='tight')
    plt.close()

radar_chart(df, score_columns)

# Distribution of Scores
plt.figure(figsize=(20, 15))
for i, metric in enumerate(score_columns, 1):
    plt.subplot(2, 3, i)
    for model in df['model_name'].unique():
        sns.kdeplot(data=df[df['model_name'] == model], x=metric, label=model)
    plt.title(f'Distribution of {metric}')
    plt.xlabel('Score')
    plt.ylabel('Density')
    if i == 3:  # Place legend outside the plots
        plt.legend(title='Model Name', bbox_to_anchor=(1.05, 1), loc='upper left')
    else:
        plt.legend([])  # Remove individual legends
plt.tight_layout()
plt.savefig('score_distributions_new_metrics.png', bbox_inches='tight')
plt.close()

# Boxplots
plt.figure(figsize=(20, 15))
for i, metric in enumerate(score_columns, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(x='model_name', y=metric, data=df)
    plt.title(metric)
    plt.xticks(rotation=45, ha='right')
    plt.xlabel('')
    if i % 3 != 1:
        plt.ylabel('')
plt.tight_layout()
plt.savefig('boxplots_new_metrics.png', bbox_inches='tight')
plt.close()

print("\nVisualizations complete. Check the generated PNG files for updated plots.")

# 7. Basic Descriptive Statistics
descriptive_stats = df.groupby('model_name')[score_columns].agg(['mean', 'std', 'min', 'max'])
print("\nDescriptive Statistics:")
print(descriptive_stats)

print("\nAnalysis complete. Check the generated PNG files for visualizations.")

Columns in the CSV file:
Index(['model_name', 'norm_type', 'variant', 'truncated_input',
       'generated_summary', 'Overall Score', 'Relevance Score',
       'Conciseness Score', 'Fluency Score', 'Accuracy Score',
       'Coherence Score', 'Overall Feedback', 'Comments on Columns'],
      dtype='object')

Using score columns: ['Overall Score', 'Relevance Score', 'Conciseness Score', 'Fluency Score', 'Accuracy Score', 'Coherence Score']

Effect Sizes (Eta Squared):
Overall Score        0.023352
Relevance Score      0.070312
Conciseness Score    0.046649
Fluency Score        0.072741
Accuracy Score       0.052493
Coherence Score      0.043075
dtype: float64

Tukey HSD Results (example for Overall Score):
                                               group1  \
0    shng2025/GPT-Valkyrie_LN-124m__AttnOnly__Billsum   
1    shng2025/GPT-Valkyrie_LN-124m__AttnOnly__Billsum   
2    shng2025/GPT-Valkyrie_LN-124m__AttnOnly__Billsum   
3    shng2025/GPT-Valkyrie_LN-124m__AttnOnly__Billsum   
4